In [ ]:
from numpy.random import choice
from string import ascii_lowercase
import re
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
con = open('war_and_peace.txt', 'r')       # load data
train = con.read()

In [ ]:
train = train.lower()                      # make lowercase
train = re.sub(r'\s+', ' ', train)         # get rid of extra whitespace and newlines

In [ ]:
train[5500:5800]

Transition Probability Matrix:

In [ ]:
letters = [i for i in ascii_lowercase]
letters.append('_')                         # _ presents any non-letter character
letters

In [ ]:
trans_matrix = pd.DataFrame(np.zeros((27, 27), dtype=int), index=letters, columns=letters)
trans_matrix

Populating Transition Probability Matrix:

In [ ]:
for i in range(len(train) - 1):

    if i % 100000 == 0:
        print('processed first %i characters out of 3210812' % i)

    if train[i] in letters:
        current_letter = train[i]                  # the preceding letter
    else:
        current_letter = '_'                       # if not letter, assign non-letter character # if not letter, assign non-letter character

    if train[i + 1] in letters:
        last_letter = train[i + 1]                  # the succeeding letter
    else:
        last_letter = '_'                           # if not letter, assign non-letter character # if not letter, assign non-letter character

    trans_matrix.loc[current_letter, last_letter] = trans_matrix.loc[current_letter, last_letter] + 1

In [ ]:
trans_matrix = trans_matrix + 1             # allowing log and any combination of letters
trans_matrix

In [ ]:
for i in range(trans_matrix.shape[0]):                   # Normalize by Row_Sum to turn into Probability
    trans_matrix.iloc[i, :] = trans_matrix.iloc[i, :] / trans_matrix.iloc[i, :].sum()

In [ ]:
sns.heatmap(trans_matrix)
plt.yticks(rotation=0)
plt.show()

In [ ]:
def decode(mapping, coded):                             # function to decode text, given mapping and coded text
    text = []
    for i in coded:
        if i in letters:
            text.append(letters[mapping.index(i)])
        else:
            text.append(letters[mapping.index('_')])
    return ''.join(text)

In [ ]:
def encode(mapping, text):                              # function to encode text, given mapping and plain text
    encoded = []
    for i in text:
        if i in letters:
            encoded.append(mapping[letters.index(i)])
        else:
            encoded.append(mapping[letters.index('_')])
    return ''.join(encoded)

In [ ]:
def loglike(decoded):                                   # function to calculate Log-like of decoded text
    loglike = 0
    last_letter = '_'
    for i in decoded:
        current_letter = i
        loglike = loglike + np.log(trans_matrix.loc[last_letter, current_letter])
        last_letter = current_letter
    loglike = loglike + np.log(trans_matrix.loc[last_letter, '_'])
    return loglike

In [ ]:
np.random.seed(12345)                          # set seed for reproducing

Randomly Scramble to create Encoding Rule (True Mapping, Pretend we dont know):

In [ ]:
mapping_true = list(choice(letters, 27, replace=False))
mapping_true

Martin Luther King Jr, Speech:

In [ ]:
correct_txt = ('Five score years ago, a great American, in whose symbolic shadow we stand today, signed the Emancipation Proclamation.'
               ' This momentous decree came as a great beacon light of hope to millions of poor slaves who had been seared in the flames'
               ' of withering injustice. It came as a joyous daybreak to end the long night of their captivity').lower()

In [ ]:
correct_txt       # decode text, and lets see if Metropolis can decipher this

In [ ]:
encoded = encode(mapping_true, correct_txt)            # Encoded based on True mapping

In [ ]:
encoded

In [ ]:
mapping0 = list(choice(letters, 27, replace=False))      # set the initial mapping
current_loglik = loglike(decode(mapping0, encoded))

max_loglik = current_loglik                              # best loglikehood so far      
max_decode = decode(mapping0, encoded)                   # best decoded text so far

In [ ]:
mapping0                 # very different from True Mapping

Metropolis Begins Here:

In [ ]:
while i <= 5000:

    tmp = choice(letters, size=2, replace=False)

    pos1, pos2 = mapping0.index(tmp[0]), mapping0.index(tmp[1])   # randomly swap 2 letters

    mapping_proposed = mapping0[:]                                # create a proposed mapping
    mapping_proposed[pos1], mapping_proposed[pos2] = mapping_proposed[pos2], mapping_proposed[pos1]

    proposed_loglik = loglike(decode(mapping_proposed, encoded))   # loglikehood of proposed mapping

    if (np.random.uniform() < np.exp(proposed_loglik - current_loglik)):   # NRG to decide accept or not
        mapping0 = mapping_proposed
        current_loglik = proposed_loglik

        if current_loglik > max_loglik:
            max_loglik = current_loglik
            max_decode = decode(mapping_proposed, encoded)

            i += 1

        print(i, decode(mapping0, encoded), '\n')